In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEndpointEmbeddings
from dotenv import load_dotenv
from langchain_core.prompts import load_prompt
from langchain_core.messages import AIMessage, HumanMessage
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


In [128]:
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id = "google/gemma-2-2b-it",
    task='text-generation',
    temperature=0.5
)

model = ChatHuggingFace(llm=llm)

embed_loader = "Qwen/Qwen3-Embedding-8B"
embed_model = HuggingFaceEndpointEmbeddings(client=embed_loader)


In [85]:
# Document Loaders
file_path = "/Users/gyan/Desktop/Faq/ugrulebook_2.pdf"
loader_ = PDFPlumberLoader(file_path)
docs = loader_.load()


In [129]:
docs[5].page_content


'PREFACE\nThe Indian Institute of Technology Bombay (IITB) is one of the Indian Institutes of Technology in\nthe country, set up with the objective of conducting research, imparting education, and training\nin various fields of Science and Technology. The Institute is now recognized as a leader in science\nand engineering education worldwide. It has established a firm foundation for education and\nresearch with a vision to be the fountainhead of new ideas and innovations in technology and\nscience. The mission of IIT Bombay is to create an ambiance in which new ideas, research and\nscholarship flourish and from which the leaders and innovators of tomorrow emerge.\nIIT Bombay on an average annually admits more than 1000 candidates for the undergraduate\nprogrammes (B.Tech./ Dual Degree and B.S.) through the Joint Entrance Examination (JEE), more\nthan 30 candidates for B.Des. Programme through the Undergraduate Common Entrance Exam\nfor Design (UCEED), around 300 candidates for the M.Sc

In [ ]:
# Function to remove \n from text
def remove_newlines(text_list):
    return [text.replace('\n', ' ') for text in text_list]


In [88]:
# Text Splitters
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap = 0
)

all_text = "\n".join(doc.page_content for doc in docs)
all_text = remove_newlines([all_text]) # removing '\n'
chunks = splitter.create_documents(all_text)
print(len(chunks))


251


In [89]:
# Vector Store
vector_store = Chroma(
    embedding_function=embed_model,
    persist_directory="Chroma_db",
    collection_name="my_sample"
)

# Delete all documents from the collection
vector_store._collection.delete(where={"id": {"$ne": ""}})

# adding documents
vector_store.add_documents(chunks)


['c8560328-d735-4249-82fe-590d7a28b567',
 '3240272d-2206-4b51-a043-7dc1d90ca269',
 '10649f6d-73a6-49e6-a666-7759c0353785',
 '61d6b167-cedf-4cb5-8247-311765409e95',
 '98105512-96fb-4660-92d4-6870ec929b4f',
 '71643f77-41a8-40bc-8b4e-eba251a74934',
 '792be6e2-b374-43ce-9baf-e0b1860fa70b',
 'be076dfb-ad8e-455d-836e-7b84025e77ad',
 'bc60791c-0730-4aa2-b9bd-59eaec2cffd7',
 '94520547-bc6d-444b-8a2d-aaa21abf11cf',
 '416d9280-941a-4f76-9fe2-acc1de46e2fa',
 '7bbac4e9-1977-4d6d-88d5-35c6456778bb',
 '48d088ad-9a71-42f0-b2ba-f1d205d86211',
 '95e0b99f-2250-4c37-b1a6-48e1e87821bb',
 'a341f062-3fec-4578-b433-b37acfbfc278',
 '09a351c0-92b4-41e6-a52f-8bb2dbe52561',
 '0ee8ae3a-4242-4373-8dcc-c404f5a237b8',
 '03fcc2e2-0469-4bc7-b5cc-0fb6eef70d61',
 'a17a1b05-09dc-47b5-9a55-0ebccacb7d14',
 '4cbec01f-cce7-4656-9b6a-d9e26f125d96',
 '5bc0963e-b9e6-4a48-99ea-f58a4094b5b2',
 '170a3906-d0e2-4b7f-acd3-81f10c3b6c2c',
 'fc736a23-9ec8-44a7-b06c-3dbcd3dd3b82',
 '9ccb9907-27dc-486f-af76-a1a1593c1328',
 '2765525e-146d-

In [90]:
prompt_2 = PromptTemplate(
            template="""You are a query rewriting assistant that helps improve user questions to make them clearer, more specific, and more answerable.

            Given the user question, rewrite the question to:

            - Use proper grammar and phrasing
            - Be more specific if needed
            - Align with the provided context
            - Remove any ambiguity
            - Ensure it helps retrieve or generate a better answer

            User Question: {question}
            Rewritten Query:""",
            input_variables=['question']
)


In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

# Contextual Compression Retrieval
base_retriever = vector_store.as_retriever(search_kwargs={"k": 10})

# Set up the compressor using an LLM
compressor = LLMChainExtractor.from_llm(llm=model)

# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

parser = StrOutputParser()

template = load_prompt(path='template.json')

parallel_chain = RunnableParallel({
    'context': prompt_2 | model | parser | compression_retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

chain = parallel_chain | template | model | parser


In [130]:
query = "what are the 6 categories in which students are categorised based on their academic performance"
chain.invoke({'question': query})


'Based on the context you provided, students are categorized into six categories:\n\n1. **Category I:**  Students with a CPI of at least 8 and no outstanding grades (FR, DX, DR, W) in a core course.\n2. **Category II:** Students with a CPI less than 8 and no outstanding grades (FR, DX, DR, W) in a core course.\n3. **Category III:** Students who have at least one outstanding (FR, DX, DR, W) grade in core courses and may have at most one FR or DX grade in any other course within two preceding regular semesters.  This condition applies as long as they have earned at least 18 credits in each semester. \n\n\nLet me know if you have other questions! \n'

In [133]:
# Manual Memory Management
chat_history = []
print(len(chat_history))

while True:
    user_input = input("You: ")
    print(f"You: {user_input}")
    chat_history.append(HumanMessage(content=user_input))

    if user_input.lower() == 'exit':
        break

    chat = "\n".join(msg.content for msg in chat_history)
    result = chain.invoke({'question': chat})
    chat_history.append(AIMessage(content=result))
    print(result)


0
You: Hi
Hi! How can I help you today? 

You: What are the main roles of faculty advisor


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-6860342d-408afa3322c4316b3d2c7b4b;3ac70a8d-0bcf-4ec0-ad20-f2f8cc328130)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [142]:
# ConversationSummaryMemory

from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

# Initialize ConversationSummaryMemory
summary_memory = ConversationSummaryMemory(
    llm=model,
    return_messages=True
)

# Setup the conversation chain using summary memory
conversation = ConversationChain(
    llm=model,
    memory=summary_memory,
    verbose=True  # Optional: helpful for debugging
)

# Interactive loop
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break

    response = conversation.predict(input=user_input)
    print(f"Assistant: {response}")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content='', additional_kwargs={}, response_metadata={})]
Human: HI
AI:

> Finished chain.
Assistant: Hey there! 👋  What's up? 😊  



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content='The human starts the conversation with "HI". The AI responds with a friendly "Hey there! 👋  What\'s up? 😊". \n', additional_kwargs={}, response_metadata={})]
Human: What is ARP
AI:

> 